In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from RDMFS_phys import HF_2RDM, MU_2RDM, GU_2RDM, BBC1_2RDM
from utils_phys import R_spectral_clean, DM2_DiagSum, twoDM_Eigvals
import scipy 

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 1.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
mol.basis = "sto-3g"
# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b

eri_h2_aa = eri_ao.copy()
eri_h2_bb = eri_ao.copy()
eri_h2_ab = eri_ao.copy()


for i in range(4):
  eri_h2_aa = np.tensordot(eri_h2_aa, C_a, axes=1).transpose(3, 0, 1, 2)
  eri_h2_bb = np.tensordot(eri_h2_bb, C_b, axes=1).transpose(3, 0, 1, 2)

eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)

h1_mo = (h1_a, h1_b)
h2_mo = (eri_h2_aa, eri_h2_ab, eri_h2_bb)

In [9]:
dm1_mo = mf.make_rdm1()
dm2 = mf.make_rdm2()

occ, C = R_spectral_clean(dm1_mo[0], h1.shape[0])
print(occ, np.sum(occ))

dm2_mo = dm2[0]+2*dm2[1]+dm2[2] # for this system HF just decides spin polarisation, to fix this I add all parts

print(dm2_mo.shape)

[1.08535919e+00 6.53157060e-01 1.28695015e-16 0.00000000e+00] 1.7385162533770555
(4, 4, 4, 4)


In [4]:
dm2_HF_mo = HF_2RDM(occ*2, M)
dm2_MU_mo = MU_2RDM(occ*2, M)
dm2_GU_mo = GU_2RDM(occ*2, M)
dm2_BBC1_mo = BBC1_2RDM(occ*2,M,mol.nelec[0])


In [5]:
print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(np.transpose(dm2_MU_mo, (0, 2, 1, 3)))), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(np.transpose(dm2_BBC1_mo, (0, 2, 1, 3)))), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(np.transpose(dm2_GU_mo, (0, 2, 1, 3)))), scipy.special.binom(4,2))

8.880517605860677 6.0
8.612722546270666 6.0
8.612722546270666 6.0
8.880517605860677 6.0


In [10]:
# Run FCI  
cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e_mo, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_mo, dm2_mo  = cisolver.make_rdm12s(ci_mo, h1.shape[0], mol.nelec)
occ, C = R_spectral_clean(dm1_mo[0], h1.shape[0])
print(occ, np.sum(occ))


[0.89413956 0.5        0.5        0.10586044] 2.0000000000000004


In [7]:
dm2_HF_mo = HF_2RDM(occ*2, M)
dm2_MU_mo = MU_2RDM(occ*2, M)
dm2_GU_mo = GU_2RDM(occ*2, M)
dm2_BBC1_mo = BBC1_2RDM(occ*2,M,mol.nelec[0])

In [8]:
print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(np.transpose(dm2_MU_mo, (0, 2, 1, 3)))), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(np.transpose(dm2_BBC1_mo, (0, 2, 1, 3)))), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(np.transpose(dm2_GU_mo, (0, 2, 1, 3)))), scipy.special.binom(4,2))

13.378616040724475 6.0
12.000000000000004 6.0
12.000000000000004 6.0
13.378616040724474 6.0
